# Capstone 1 Project: Data Wrangling

Focus of exploratory data analysis will be;
- Missing data (Null values)
- Invalid data
- Duplicates
- Inconsistent column naming
- Untidiness
- Any further processing of data to make it meaningful
- Outliers

The crop information is available with variety of different data points but we will focus mainly on Yield and how it is affected by the Arces harvested, climate and labor avalability. 
<p><b>Acres Harvested</b>: Total strech of land from which the crop is harvested measured in arces. Where 1 acres is 43560 sq. ft.</p>
<p><b>Production</b>: The total amount of crop produce by weight measured in Bushel (imperical unit).<p>
<p><b>Yield</b>: Yield is the output of the land per acres, so it is production per acres (Production/Acres Harvested)</p>

## Importing data

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from __future__ import print_function

In [3]:
data_file = 'Data/USDA_Data_v2.0.xlsx'
data_xl_sheets = pd.ExcelFile(data_file).sheet_names
corn_df = pd.read_excel(data_file, sheet_name='Corn')[['Year','State','County','Data Item', 'Value']]
barley_df = pd.read_excel(data_file, sheet_name='Barley')[['Year','State','County','Data Item','Value']]
peanuts_df = pd.read_excel(data_file, sheet_name='Peanuts')[['Year','State','County','Data Item','Value']]
soybean_df = pd.read_excel(data_file, sheet_name='Soybean')[['Year','State','County','Data Item','Value']]
price_df = pd.read_excel(data_file, sheet_name='Price Received')[['Year','State','Commodity','Value']]
# #importing unemployment rate
# unemployment_df = pd.read_excel(data_file, sheet_name='US UnemploymentRate')
# #setting case to upper, easier to merge with rest of the data.
# unemployment_df['State']=unemployment_df['State'].str.upper()

Even though the data we got downloaded from USDA website was as County level, it still seem to have another level of detail in it. After checking the raw data the level below County is 'Ag District'. We can ignore the column but expect duplicate rows for each County for a Year. This needs to be aggregated to County level.

In [4]:
corn_df[corn_df.duplicated(['Year','State','County','Data Item'],keep=False)].sort_values(['Year','State','County','Data Item']).head()

,Year,State,County,Data Item,Value
203655,1984,ALABAMA,OTHER (COMBINED) COUNTIES,"CORN, GRAIN - ACRES HARVESTED",1500.0
203781,1984,ALABAMA,OTHER (COMBINED) COUNTIES,"CORN, GRAIN - ACRES HARVESTED",900.0
203656,1984,ALABAMA,OTHER (COMBINED) COUNTIES,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",75000.0
203782,1984,ALABAMA,OTHER (COMBINED) COUNTIES,"CORN, GRAIN - PRODUCTION, MEASURED IN BU",35000.0
203657,1984,ALABAMA,OTHER (COMBINED) COUNTIES,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",50.0


For most of the crops we have aggregate data but for corn only the most widely produced variety has data about Acres Harvested, Production and Yield. Hence in case of Corn we will be using only the data of grain variety of corn.

In [5]:
corn_df['Data Item'].unique()

array([u'CORN, GRAIN - ACRES HARVESTED',
       u'CORN, GRAIN - PRODUCTION, MEASURED IN BU',
       u'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE'], dtype=object)

### Tidy up
Now we want to pivot the values in Data Item in thier own columns, so each row represnts only a obervation.

But before that we need to shorten up values so that column names are short enough. To do that we do the following,
1. Remove some of the redundent information like 'CORN, GRAIN'.
2. Certain information from Data Item needs to be cleaned up.

In [6]:
#CORN
#Removing any combination of CORN GRAIN and preceding whitespaces
corn_df.loc[:,'Data Item'] = (corn_df['Data Item'].str.replace(r'^CORN, GRAIN[\s,][\s-]','')).str.replace(r'^\s','')

#Removing anything after comma(,) as it just tells you the unit of the value.
corn_df.loc[:,'Data Item'] = corn_df['Data Item'].str.split(',').str[0]

#----------------------------------------------------------
#BARLEY
#Removing 'BARLEY -' and whitespaces
barley_df.loc[:,'Data Item'] = (barley_df['Data Item'].str.replace(r'^BARLEY[\s,][\s-]','')).str.replace(r'^\s','')

#Removing anything after comma(,) as it just tells you the unit of the value.
barley_df.loc[:,'Data Item'] = barley_df['Data Item'].str.split(',').str[0]

#----------------------------------------------------------
#PEANUTS
#Removing 'PEANUTS -' and whitespaces
peanuts_df.loc[:,'Data Item'] = (peanuts_df['Data Item'].str.replace(r'^PEANUTS[\s,][\s-]','')).str.replace(r'^\s','')

#Removing anything after comma(,) as it just tells you the unit of the value.
peanuts_df.loc[:,'Data Item'] = peanuts_df['Data Item'].str.split(',').str[0]

#----------------------------------------------------------
#SOY BEANS
#Removing 'SOYBEANS -' and whitespaces
soybean_df.loc[:,'Data Item'] = (soybean_df['Data Item'].str.replace(r'^SOYBEANS[\s,][\s-]','')).str.replace(r'^\s','')

#Removing anything after comma(,) as it just tells you the unit of the value.
soybean_df.loc[:,'Data Item'] = soybean_df['Data Item'].str.split(',').str[0]

print("{A}\n\n{B}\n\n{C}\n\n{D}".format(A=corn_df.head(),B=barley_df.head(),C=peanuts_df.head(),D=soybean_df.head()))

   Year    State   County        Data Item     Value
0  2017  ALABAMA  AUTAUGA  ACRES HARVESTED     650.0
1  2017  ALABAMA  AUTAUGA       PRODUCTION  111000.0
2  2017  ALABAMA  AUTAUGA            YIELD     170.8
3  2017  ALABAMA  BULLOCK  ACRES HARVESTED     800.0
4  2017  ALABAMA  BULLOCK       PRODUCTION  108000.0

   Year       State                     County        Data Item     Value
0  2018  CALIFORNIA  OTHER (COMBINED) COUNTIES  ACRES HARVESTED    1700.0
1  2018  CALIFORNIA  OTHER (COMBINED) COUNTIES       PRODUCTION  100000.0
2  2018  CALIFORNIA  OTHER (COMBINED) COUNTIES            YIELD      58.8
3  2018  CALIFORNIA            SAN LUIS OBISPO  ACRES HARVESTED    7800.0
4  2018  CALIFORNIA            SAN LUIS OBISPO       PRODUCTION  450000.0

   Year    State                     County        Data Item    Value
0  2017  ALABAMA                     DALLAS  ACRES HARVESTED     1600
1  2017  ALABAMA                     DALLAS       PRODUCTION  5500000
2  2017  ALABAMA          

Pivoting the 'Data Item' as well as aggregating values at County level.

In [7]:
#CORN
pivoted_corn_df = corn_df.pivot_table(index=['Year','State','County'], columns='Data Item', values='Value',aggfunc='sum').reset_index()
pivoted_corn_df.columns.name=None
pivoted_corn_df.loc[:,'CROP']='CORN'

#----------------------------------------------------------
#BARLEY
pivoted_barley_df = barley_df.pivot_table(index=['Year','State','County'], columns='Data Item', values='Value',aggfunc='sum').reset_index()
pivoted_barley_df.columns.name=None
pivoted_barley_df.loc[:,'CROP']='BARLEY'

#----------------------------------------------------------
#PEANUTS
pivoted_peanuts_df = peanuts_df.pivot_table(index=['Year','State','County'], columns='Data Item', values='Value',aggfunc='sum').reset_index()
pivoted_peanuts_df.columns.name=None
pivoted_peanuts_df.loc[:,'CROP']='PEANUTS'

#----------------------------------------------------------
#SOY BEANS
pivoted_soybean_df = soybean_df.pivot_table(index=['Year','State','County'], columns='Data Item', values='Value',aggfunc='sum').reset_index()
pivoted_soybean_df.columns.name=None
pivoted_soybean_df.loc[:,'CROP']='SOYBEANS'

#----------------------------------------------------------
#PRICE
pivoted_price_df=price_df.copy()
pivoted_price_df.columns = ['Year','State','CROP','PRICE']
#A little clean up
pivoted_price_df['PRICE'] = pivoted_price_df['PRICE'].replace(" (NA)",np.NaN)

print("{A}\n\n{B}\n\n{C}\n\n{D}\n\n{E}".format(A=corn_df.head(),B=barley_df.head(),C=peanuts_df.head(),D=soybean_df.head(),E=pivoted_price_df.head()))

   Year    State   County        Data Item     Value
0  2017  ALABAMA  AUTAUGA  ACRES HARVESTED     650.0
1  2017  ALABAMA  AUTAUGA       PRODUCTION  111000.0
2  2017  ALABAMA  AUTAUGA            YIELD     170.8
3  2017  ALABAMA  BULLOCK  ACRES HARVESTED     800.0
4  2017  ALABAMA  BULLOCK       PRODUCTION  108000.0

   Year       State                     County        Data Item     Value
0  2018  CALIFORNIA  OTHER (COMBINED) COUNTIES  ACRES HARVESTED    1700.0
1  2018  CALIFORNIA  OTHER (COMBINED) COUNTIES       PRODUCTION  100000.0
2  2018  CALIFORNIA  OTHER (COMBINED) COUNTIES            YIELD      58.8
3  2018  CALIFORNIA            SAN LUIS OBISPO  ACRES HARVESTED    7800.0
4  2018  CALIFORNIA            SAN LUIS OBISPO       PRODUCTION  450000.0

   Year    State                     County        Data Item    Value
0  2017  ALABAMA                     DALLAS  ACRES HARVESTED     1600
1  2017  ALABAMA                     DALLAS       PRODUCTION  5500000
2  2017  ALABAMA          

### Standardizing all values to same unit Production in pounds (lbs) and price in \$/pound(lb) 

In [8]:
# The unit of measurement of some crops (CORN, BARLEY and SOYBEAN) is in BU, while peanuts are in lb. 
# According below information even the number of pounds(lb) in an BU, varies from crop to crop. Since we will convert all the production in lb.
# https://grains.org/markets-tools-data/tools/converting-grain-units/

pivoted_corn_df.loc[:,'PRODUCTION'] = pivoted_corn_df['PRODUCTION']*56
pivoted_barley_df.loc[:,'PRODUCTION'] = pivoted_barley_df['PRODUCTION']*48
pivoted_soybean_df.loc[:,'PRODUCTION'] = pivoted_soybean_df['PRODUCTION']*60

#Converting price from $/BU to $/lb
pivoted_price_df.loc[pivoted_price_df['CROP']=='CORN','PRICE']/=56
pivoted_price_df.loc[pivoted_price_df['CROP']=='BARLEY','PRICE']/=48
pivoted_price_df.loc[pivoted_price_df['CROP']=='SOYBEANS','PRICE']/=60

#Filling nulls
pivoted_price_df.loc[:,'PRICE'] =pivoted_price_df.groupby(['Year','CROP'])['PRICE'].transform(lambda x: x.fillna(x.mean()))
pivoted_price_df.head()

,Year,State,CROP,PRICE
0,2017,ALABAMA,CORN,0.072143
1,2017,ALABAMA,PEANUTS,0.221000
2,2017,ALABAMA,SOYBEANS,0.157167
3,2017,ALASKA,BARLEY,0.109375
4,2017,ALASKA,CORN,0.066842


### Combining datasets

In [9]:
#Combining all crops into single dataframe
crop_df = pd.concat([pivoted_corn_df,pivoted_barley_df,pivoted_peanuts_df,pivoted_soybean_df]).reset_index().drop(columns='index')

#Recalculating Yield, as Sum is not an appropriate aggregation for it.
crop_df.loc[:,'YIELD']=crop_df['PRODUCTION']/crop_df['ACRES HARVESTED']

#Adding prices
crop_df = pd.merge(crop_df, pivoted_price_df, how='left')

#Rearranging columns
crop_df = crop_df[['CROP','Year','State','County','ACRES HARVESTED','PRODUCTION','YIELD','PRICE']]
crop_df.tail()

,CROP,Year,State,County,ACRES HARVESTED,PRODUCTION,YIELD,PRICE
147039,SOYBEANS,2017,WISCONSIN,WAUKESHA,21100.0,56280000.0,2667.298578,0.155667
147040,SOYBEANS,2017,WISCONSIN,WAUPACA,29800.0,82860000.0,2780.536913,0.155667
147041,SOYBEANS,2017,WISCONSIN,WAUSHARA,15300.0,38460000.0,2513.725490,0.155667
147042,SOYBEANS,2017,WISCONSIN,WINNEBAGO,43100.0,122340000.0,2838.515081,0.155667
147043,SOYBEANS,2017,WISCONSIN,WOOD,20100.0,51780000.0,2576.119403,0.155667


### Clean up

Looking for any odd values in these columns.

In [10]:
sp_ch = ['`','~','!','@','#','$','%','^','&','*','(',')','_','-','+','=','{','[','}','}','|','\\',':',';','"',"'",'<',',','>','.','?','/']
esc_lst =[re.escape(s) for s in sp_ch]
pattern = '|'.join(esc_lst)
print('County:',crop_df[crop_df['County'].str.contains(pattern, case=False)]['County'].unique())
print('State:',crop_df[crop_df['State'].str.contains(pattern, case=False)]['State'].unique())
print('Year:',crop_df[(crop_df['Year']<1984) | (crop_df['Year']>2018)]['Year'].unique())
print('Negative Values:',crop_df[(crop_df<0).any(1)])

County: [u'OTHER (COMBINED) COUNTIES' u'ST. JOHNS' u'ST. JOSEPH' u'ST. LOUIS'
 u'STE. GENEVIEVE' u'ST. JOHN THE BAPTIST']
State: []
Year: []
Negative Values: Empty DataFrame
Columns: [CROP, Year, State, County, ACRES HARVESTED, PRODUCTION, YIELD, PRICE]
Index: []


There aren't any odd values in any of the columns but one value that can be of concern during modeling is "OTHER (COMBINED) COUNTIES". This is because mixture of combined counties changes for each year and crop. The range of values for Area, Production and Yield is so large that it will throw off the model for this County value. Hence we will drop it.

In [11]:
print(crop_df[crop_df['County']=='OTHER (COMBINED) COUNTIES'].groupby('CROP').describe().loc[:,(slice(None),['max','min'])])
filtered_crop_df = crop_df[~(crop_df['County']=='OTHER (COMBINED) COUNTIES')]

         ACRES HARVESTED                PRICE           PRODUCTION  \
                     min        max       min       max        min   
CROP                                                                 
BARLEY              50.0   683700.0  0.022917  0.140000   120000.0   
CORN               100.0  1668500.0  0.025893  0.141071   224000.0   
PEANUTS              5.0   224200.0  0.164000  0.431000    11000.0   
SOYBEANS            30.0  1763540.0  0.067500  0.245000    54000.0   

                              YIELD                  Year          
                   max          min           max     min     max  
CROP                                                               
BARLEY    2.115163e+09   426.666667   7609.127517  1984.0  2018.0  
CORN      1.305159e+10  1536.000000  13864.592953  1984.0  2017.0  
PEANUTS   9.232500e+08   876.470588   4623.287671  1984.0  2017.0  
SOYBEANS  4.314090e+09   540.000000   3594.381506  1984.0  2017.0  


Looks like all of the columns have valid data types that corresponds to the value expected in the column.
But looking for the non-nulls in the key columns ACRES HARVESTED,PRODUCTION and YIELD have good population except for few nulls. Which we will deal with soon. For rest of the columns where non-null values that are not even 20%, we will have to ignore that data (drop columns).

In [12]:
print(filtered_crop_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144424 entries, 0 to 147043
Data columns (total 8 columns):
CROP               144424 non-null object
Year               144424 non-null int64
State              144424 non-null object
County             144424 non-null object
ACRES HARVESTED    143870 non-null float64
PRODUCTION         143870 non-null float64
YIELD              143869 non-null float64
PRICE              144340 non-null float64
dtypes: float64(4), int64(1), object(3)
memory usage: 9.9+ MB
None


In [13]:
# Looking above Nulls are in any of the numerical columns.
# Dropping nulls if Harvested Acres or Production is missing, as these are important features. 
# Using statistical method to get approximate values may skew the model, hence dropping these rows is a better option.
temp_crop_df = filtered_crop_df.dropna(axis=0,subset=['ACRES HARVESTED','PRODUCTION'])

#Looking for pattern in the remaining nulls. Looks like the price is missing for Barley for the year 2018.
# We can drop these too. Few values 
print('Pattern for Nulls: {}'.format(pd.unique(temp_crop_df[temp_crop_df.isnull().any(axis=1)][['CROP','Year']].values.ravel('K'))))
print('What are other values does the above pattern have for Price: {}'.\
      format(temp_crop_df.loc[(temp_crop_df['CROP']=='BARLEY') & (temp_crop_df['Year']==2018),:]['PRICE'].unique()))

clean_crop_df = temp_crop_df.dropna(axis=0,subset=['PRICE'])
print('Any more nulls?\n {}'.format(clean_crop_df.isnull().any()))

Pattern for Nulls: ['BARLEY' 2018L]
What are other values does the above pattern have for Price: [nan]
Any more nulls?
 CROP               False
Year               False
State              False
County             False
ACRES HARVESTED    False
PRODUCTION         False
YIELD              False
PRICE              False
dtype: bool


Looking for duplicates by Crop, Year, State and County but there are none.

In [14]:
clean_crop_df[clean_crop_df.duplicated(['CROP','Year','State','County'],keep=False)]
clean_crop_df.shape

(143785, 8)

The distribution of all the columns looks normally distributed. Although Acres Harvested and Production is denser towards the 0, which just mean there are more smaller farms. Hence no further action is required here.

In [15]:
# import seaborn as sns
# #clean_crop_df[['ACRES HARVESTED','PRODUCTION','YIELD','PRICE']].hist(figsize=(15,25),layout=(4,1),bins=100, column=clean_crop_df['CROP'])
# sns.distplot(clean_crop_df, kde=False, rug=True)
# plt.show()

****************

## Unemployment Rate
This dataset is from Bureau of Labor Statistics. The downloaded excel workbook consisted of state wise unemployment for every month since Jan 1976 to Nov 2018. The set was split by state with each state in a separate sheet, hence needed to be combined into one large set. And since the crop statistics is on yearly basis, this set needs to be aggregated to yearly numbers.

Once the set is combined and aggregated we write it into another excel file and later manually added as a seperate sheet to rest of the data workbook.

In [16]:
xl_file = 'Data/DataFinder-20190114232211.xlsx'
xl_sheets = pd.ExcelFile(xl_file).sheet_names
unemployment_df = pd.DataFrame({})

def read_unemployment_sheet(sheet):
    #Reading one sheet from excel, while only selecting first 5 columns as rest of the columns are empty. 
    df = pd.read_excel(xl_file, sheet_name=sheet).iloc[:,:5]
    
    #Assigning state name, that's located at 7th row of 2nd column, to seperate column.
    state_name = df.iloc[6,1]
    #print(state_name)
    df['Unnamed: 4']=state_name
    
    #Filtering out all rows above the actual data.
    df = df.iloc[11:,:]
    
    #Renaming columns
    df.columns = ['Year','Period','Month','Unemployment Rate','State']
    
    #Type casting to make sure 'Unemployment Rate' is numeric before taking yearly averages
    df['Unemployment Rate']=pd.to_numeric(df['Unemployment Rate'])
    
    #Takes mean 'Unemployment Rate' across year.
    df['Yearly Unemployment Rate'] = df.groupby('Year')['Unemployment Rate'].transform('mean')
    
    #returns selected columns and dropping duplicates.
    return df[['State','Year','Yearly Unemployment Rate']].drop_duplicates()
    
unemployment_df = unemployment_df.append(map(read_unemployment_sheet,xl_sheets))
unemployment_df['State']=unemployment_df['State'].str.upper()
unemployment_df['Year']=unemployment_df['Year'].astype('int64')
print(unemployment_df.head())
# writer = pd.ExcelWriter('Data/US_Historical_UnemploymentRate_ByState.xlsx')
# xl_df.to_excel(writer,'US UnemploymentRate', index=False)
# writer.save()

      State  Year  Yearly Unemployment Rate
11  ALABAMA  1976                  6.766667
23  ALABAMA  1977                  7.316667
35  ALABAMA  1978                  6.316667
47  ALABAMA  1979                  7.175000
59  ALABAMA  1980                  8.883333


Combining Unemployment data with yield.

In [17]:
unemp_crop_df = pd.merge(clean_crop_df,unemployment_df,how='left')
unemp_crop_df.head()

,CROP,Year,State,County,ACRES HARVESTED,PRODUCTION,YIELD,PRICE,Yearly Unemployment Rate
0,CORN,1984,ALABAMA,AUTAUGA,2500.0,5768000.0,2307.200000,0.050893,11.0
1,CORN,1984,ALABAMA,BALDWIN,40700.0,178976000.0,4397.444717,0.050893,11.0
2,CORN,1984,ALABAMA,BARBOUR,7700.0,24304000.0,3156.363636,0.050893,11.0
3,CORN,1984,ALABAMA,BLOUNT,2200.0,6944000.0,3156.363636,0.050893,11.0
4,CORN,1984,ALABAMA,BULLOCK,2000.0,6048000.0,3024.000000,0.050893,11.0


Exporting to Excel file.

In [18]:
corp_writer = pd.ExcelWriter('Data/Crop_Pivoted.xlsx')
unemp_crop_df.to_excel(corp_writer,'Crop Data', index=False)
corp_writer.save()